In [1]:

from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.chroma import Chroma

from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
caminhos = [
    "files/atom.pdf",
    "files/python.pdf"
]

paginas = []
for caminho in caminhos:
    loader = PyPDFLoader(caminho)
    paginas.extend(loader.load())

recur_split = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50,
    separators= ["\n\n", "\n", ".", " ", ""]
)

documentos = recur_split.split_documents(paginas)

for i, doc in enumerate(documentos):
    doc.metadata["source"] = doc.metadata["source"].replace("/", "")
    doc.metadata["doc_id"] = i


In [3]:
import tempfile

diretorio = tempfile.mkdtemp()

embeddings_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)


/tmp/ipykernel_66636/2813969724.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = HuggingFaceEmbeddings(
/home/lucas.abner/miniconda3/envs/agente-pdf/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-18 14:42:56.339692: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752860576.403872   66636 cuda_dnn

In [20]:
vectordb = Chroma.from_documents(
    documents=documentos,
    embedding=embeddings_model,
    persist_directory=diretorio
)

In [21]:
from langchain_ollama.chat_models import ChatOllama

llm = ChatOllama(
    model="mistral:latest"
)

In [22]:
from langchain.chains.retrieval_qa.base import RetrievalQA

chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectordb.as_retriever(search_type="mmr"),
    verbose=True
)

In [36]:
pergunta = "Como funciona o IF/ELSE?"

print(chain.invoke({"query": pergunta})["result"].replace(".", "\n"))



> Entering new RetrievalQA chain...

> Finished chain.
 O `IF/ELSE` é uma estrutura de controle de fluxo em programação que permite a execução de códigos condicionalmente, baseado no resultado de uma comparação ou teste lógico
 A sintaxe básica do `IF` é:

```python
if condição_de_teste:
    # Código a ser executado se for verdadeira a condição de teste
else:
    # Código a ser executado se for falsa a condição de teste
```

O exemplo seguinte mostra um uso básico do `IF/ELSE` em Python:

```python
idade = 18
if idade >= 18:
    print("Você é adulto")
else:
    print("Você não é adulto")
```

Neste exemplo, se a variável `idade` for maior ou igual a 18, então será impresso "Você é adulto", caso contrário será impresso "Você não é adulto"



In [8]:
from langchain.prompts import PromptTemplate

chain_prompt = PromptTemplate.from_template(
    """Utilize o contexto fornecido para responder a pergunta ao final.
    Se você não sabe a resposta, apenas diga que não sabe e não invente uma resposta.
    Utilize três frases no máximo, mantenha a resposta concisa.

    Contexto: {context}

    Pergunta: {question}

    Resposta:
"""
)

In [9]:
from langchain.chains.retrieval_qa.base import RetrievalQA

chat_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectordb.as_retriever(search_type="mmr"),
    chain_type_kwargs={"prompt": chain_prompt},
    return_source_documents=True,
    verbose=True
)

In [34]:
pergunta = "Abstract"

result = chat_chain.invoke({"query": pergunta})




> Entering new RetrievalQA chain...

> Finished chain.


In [35]:
print(result["result"].replace(".", "\n"))

 Inheritance allows for the creation of child classes that inherit properties and methods from a parent class, simplifying complex code structures in constrained machine learning architectures
 The __init__() function is crucial to initialize objects with specific attributes when defining a new class



In [14]:
result["source_documents"]

[Document(metadata={'author': '', 'creationdate': '2015-10-13T00:36:11+00:00', 'creator': 'LaTeX with hyperref package', 'doc_id': 9, 'keywords': '', 'moddate': '2015-10-13T00:36:11+00:00', 'page': 1, 'page_label': '2', 'producer': 'pdfTeX-1.40.12', 'ptex.fullbanner': 'This is pdfTeX, Version 3.1415926-2.3-1.40.12 (TeX Live 2011) kpathsea version 6.0.1', 'source': 'filesatom.pdf', 'subject': '', 'title': '', 'total_pages': 11, 'trapped': '/False'}, page_content='constrained machine learning architectures. Chemical groups are deﬁned by the spatial arrangement\nand bonding of multiple of atoms in space, but these atoms are proximate to each other. When\nchemical groups interact, e.g. through hydrogen bonding or π-bond stacking, the strength of their\nrepulsion or attraction may vary with their type, distance, and angle, but these are predominantly\nlocal effects [8]. More complex bioactivity features may be described by considering neighboring'),
 Document(metadata={'author': '', 'creati

In [15]:
from langchain.globals import set_debug

set_debug(True)

pergunta = "Smile de pyridine N-oxide, do methylpyrrole e do cubane"
resposta = chat_chain.invoke({"query": pergunta})

set_debug(False)

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Smile de pyridine N-oxide, do methylpyrrole e do cubane"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "Smile de pyridine N-oxide, do methylpyrrole e do cubane",
  "context": "[28] G. W. Bemis and M. A. Murcko, “The properties of known drugs. 1. molecular frameworks,”\nJournal of Medicinal Chemistry, vol. 39, no. 15, pp. 2887–2893, 1996.\n10\n\n0.0 0.2 0.4 0.6 0.8 1.0\nFP Rate\n0.0\n0.2\n0.4\n0.6\n0.8\n1.0\nTP Rate\n1qzy (0.76)\n10-3 10-2 10-1 100\nFP Rate\n0.0\n0.2\n0.4\n0.6\n0.8\n1.0\nTP Rate\n1qzy (0.44)\n0.0 0.2 0.4 0.6 0.8 1.0\nFP Rate\n0.0\n0.2\n0.4\n0.6\n0.8\n1.0\nTP Rate\n3bej (0.74)\n10-3 10-2 10-1 100\nFP Rate\n0.0\n0.2\n0.4\n0.6\n0.8\n1.0\nTP Rate\n3bej (0.24)\n0.0 0.2 0.4 0.6 0.8 1.0\nFP Rate\n0.0\n0.2\n0.4\n0.6\n0.

In [16]:
from langchain.chains.retrieval_qa.base import RetrievalQA

chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectordb.as_retriever(search_type="mmr"),
    chain_type="refine",
    return_source_documents=True
)

pergunta = "VH S85N+R87T"
resposta = chain.with_config({"temperature": 0}).invoke({"query": pergunta})
print(resposta["result"])
print(resposta["source_documents"])

KeyboardInterrupt: 

{'query': 'VH S85N+R87T',
 'result': " In the provided VH S85N+R87T residues context, it is essential to mention that these mutations are part of a group of three stabilizing double-mutants in an antibody (VH) structure. The other two mutant pairs are A40R+K43D and R50S+R59N, which were prioritized due to consistent Rosetta results between single and double mutant scans.\n\nThese triple mutations were given priority because the models' probabilities for these specific substitutions were obtained from investigated language models. The SMILES notation is a text-based representation of molecules used in computational chemistry, and it was not directly related to the VH residue mutation context provided."}